In [ ]:
import datajoint as dj
import sys
sys.path.append('/Users/carlos/Github/UberPhys/src')

import numpy as np

import sys
import pickle
# sys.path.append('/Users/carlos/Github/UberPhys/src')
sys.path.append('/Users/carlos/Github/bl_pipeline_python')
import datajoint as dj
import utility.blob_transformation as bt
import os


dj.blob.use_32bit_dims = True  # necessary for pd.blob read

dj.config['database.host'] = 'datajoint01.pni.princeton.edu'
# dj.config['database.host'] = 'bdata00.pni.princeton.edu'
dj.config['database.user'] = 'brody'  # enter your puid username

dj.conn()

if dj.config['database.host'] != 'datajoint01.pni.princeton.edu':
    raise ValueError("Only run this on datajoint01.pni.princeton.edu")


In [ ]:
schema = dj.schema('brody_test')

@schema
class BehaviorEvent(dj.Manual):
     definition = """
     sessid:                 INT(11)                    # Unique number for session
     id_event:               INT(11)                    # Unique number for event                      
     -----
     trial:                  INT(10)                    # trial number in session
     event_type:             VARCHAR(16)                # type of event in session (e.g. pokes, states)
     event_name:             VARCHAR(32)                # sub category of event type (e.g. C, L, R, state0)
     entry_num:              INT(10)                    # occurence number of event inside trial
     in_time=null:           DOUBLE                     # start time of event
     out_time=null:          DOUBLE                     # end time of event
     """






## Load pickled data and insert it into BehaviorEvent()

Safe to run, it'll only insert sessions that aren't already in there.

In [ ]:
ratname = "A327"
fp = open("/Users/carlos/Github/bl_pipeline_python/carlos/%s_sessionsAndBevs.pkl" % ratname, "rb")
sessionsAndBevs = pickle.load(fp)


for i in range(len(sessionsAndBevs)):
    if len(sessionsAndBevs[i]) == 0:  # if not events, skip session
        continue
    existing = (BehaviorEvent & "sessid=%d" % sessionsAndBevs[i][0]["sessid"]).fetch("id_event")
    if len(existing) > 0:  # if session already exists, skip
        print("sessid=%d (i=%d/%d) already exists" % (sessionsAndBevs[i][0]["sessid"], i, len(sessionsAndBevs)))
        continue
    
    # Ok, insert this session:
    BehaviorEvent.insert(sessionsAndBevs[i])

    # report every now and then
    if i % 10 == 0:
        print("Inserted %d/%d sessions" % (i, len(sessionsAndBevs)))

In [ ]:
sessionsAndBevs[i]

In [ ]:
gu = (BehaviorEvent() & "sessid=%d" % sessionsAndBevs[1][0]["sessid"]).fetch(as_dict=True)

In [ ]:
gu = (BehaviorEvent() & "sessid=%d" % 23).fetch(as_dict=True)

In [ ]:
gu

In [ ]:
bdata = dj.create_virtual_module('bdata', 'bdata')

In [ ]:
def matStruct2py(m, verbose=False):
    """
    This function takes a dj.blob.MatStruct object and returns a dictionary 
    with its contents of the Matlab file.

    Parameters
    ----------
    """

    def cellParse(m):
        """
        This function parses a cell array. It is called by singleLevelParse() and structureParse().
        For cell arrays with a single element, we'll try to unpack the content into a regular data type.
        For cell arrays with multiple elements, the different elements coule be different types, so
        they'll go into a numpy object array (dtype = np.object_) to allow for different data types.

        Parameters
        ----------
        m : np.array

        Returns
        -------
        res : np.array
            The parsed contents of the cell array.

        goodFlag : bool
            True if the parsing of all elements was successful, False otherwise.


        """
        if m.size == 1: # cell has only one element, we can return it as a regular data type if possible
            res, goodFlag = singleLevelParse(m.flat[0])
            if (not goodFlag) and verbose:
                print("Warning: singleLevelParse() returned with error when object to be parsed was ", m.flat[i])
            return res.squeeze()
        
        # else pack contents into a cell to allow different data types for different entries:    
        res = np.empty(m.shape, np.object_)
        for i in range(m.size):
            res.flat[i], goodFlag = singleLevelParse(m.flat[i])
            if (not goodFlag) and verbose:
                print("Warning: singleLevelParse() returned with error when object to be parsed was ", m.flat[i])
        return res.squeeze()


    def singleLevelParse(m):
        """
        This function parses a single element. It is called by cellParse() and structureParse(),
        and can in turn call them, to handle nested structures and cells.
        If the passed element has a data type with length greater than one, then 
        unless it is MatlabOpaque, it is a structure, so structureParse() is called to parse it.
        If its data type is a subtype of np.object_, then it is a cell array, 
        so cellParse() is called to parse it. Otherwise it is parsed as a regular data type, with
        attempts to unsqueeze as far as possible (e.g., a 1x1 array is converted to a scalar).

        Parameters
        ----------
        m : np.array

        Returns
        -------
        res : np.array
            The parsed contents of m.

        goodFlag : bool
            True if the parsing was successful, False otherwise.
        """
        goodFlag = True    
        if hasattr(m, 'dtype') and len(m.dtype)>1:
            if type(m) == scipy.io.matlab.MatlabOpaque:
                res = "Sorry, cannot parse a MatlabOpaque object"
                if verbose:
                    print("Warning: singleLevelParse() found a MatlabOpaque object")
                goodFlag = False
            else:
                res, goodFlag = structureParse(m)
                if (not goodFlag) and verbose:
                    print("Warning: structureParse() returned with error when object to be parsed was ", m)
        elif not hasattr(m, 'dtype'):
            res = None
            goodFlag = False
            if verbose:
                print("Warning: singleLevelParse() found no dtype attribute for entry ", m)
        elif np.issubdtype(m.dtype, np.object_):
            res = cellParse(m)
        elif np.issubdtype(m.dtype, np.integer) and m.size == 1:
            res = m[0,0].astype(int).squeeze()
        elif np.issubdtype(m.dtype, np.integer) and (m.size == 0 or m.size > 1):
            res = m.astype(int).squeeze()
        elif np.issubdtype(m.dtype, np.float_) and m.size == 1:
            res = m[0,0].astype(float).squeeze()
        elif np.issubdtype(m.dtype, np.float_) and (m.size == 0 or m.size > 1):
            res = m.astype(float).squeeze()
        elif np.issubdtype(m.dtype, np.float32) and m.size == 1:
            res = m[0,0].astype(np.float32).squeeze()
        elif np.issubdtype(m.dtype, np.float32) and (m.size == 0 or m.size > 1):
            res = m.astype(np.float32).squeeze()
        elif m.dtype.descr[0][1].startswith('<U') and m.size == 1:
            res = m[0].squeeze()
        elif m.dtype.descr[0][1].startswith('<U') and (m.size == 0 or m.size > 1):
            res = m.astype(str).squeeze()
        else:
            res = None
            if verbose:
                print("Warning: singleLevelParse() coudn't parse, it is missing a case for ", m)
            goodFlag = False

        return res, goodFlag


    def structureParse(m):
        """
        This function parses a structure. It is called by singleLevelParse().
        It returns a dictionary with the contents of the structure.

        Parameters
        ----------
        m : np.array

        Returns
        -------
        res : dict
            The parsed contents of the structure.
        """
        allGood = True
        res = {}
        z = m.dtype.names
        for n in range(len(z)):
            if len(m.shape)==1:
                msg = m[0][z[n]]
            elif len(m.shape)==2:
                msg = m[0,0][z[n]]
            else:
                # print("Warning: structureParse() found unexpected shape for entry ", k)
                raise ValueError("Unexpected shape for entry ", m)

            if hasattr(msg, 'dtype'):
                if len(msg.dtype)<=1:
                    res[z[n]], goodFlag = singleLevelParse(msg)
                    allGood = allGood and goodFlag
                    if (not goodFlag) and verbose:
                        print("singleLevelParse() just returned with error when msg was ", msg)
                else:   
                    if type(msg) == scipy.io.matlab.MatlabOpaque:
                        res[z[n]] = "Sorry, cannot parse a MatlabOpaque object"
                        if verbose:
                            print("Warning: singleLevelParse() found a MatlabOpaque object")
                        allGood = False
                    else:
                        res[z[n]], goodFlag = structureParse(msg)
                        allGood = allGood and goodFlag
                        if (not goodFlag) and verbose:
                            print("structure parse() just returned with error when msg was ", msg)
            else:
                if verbose:
                    print("structureParse(): Skipping ", z[n])
                allGood = False

        return res, allGood

    res = {}
    skipped = []
    for k, v in m.items():
        if not k.startswith("__"):
            if hasattr(m[k], 'dtype'):
                res[k], goodFlag = singleLevelParse(m[k])
                if (not goodFlag) and verbose:
                    print("Warning: singleLevelParse() returned with error at outer level when k was ", k)
            else:
                if verbose:
                    print("Outer level Skipping ", k)
                skipped.append[v]

    return res


In [ ]:
mysessid = 956289
gu = (bdata.ParsedEvents & "sessid = %d" % mysessid).fetch('peh', as_dict=True)[0]['peh']

In [ ]:
gaz = 2

def recArrayParsed(m):
    if m.size > 1:
        m = m.squeeze()

    names = m.dtype.names
    nNames = len(names)
    nEntries = m.size

    res = np.empty(nEntries, np.object_)

    for entry in range(nEntries):
        res[entry] = {} # the dict for this trial, with the names as keys
        for n in range(nNames):
            res[entry][names[n]] = {}
            z = m[entry][names[n]][0]
            if z.size > 0:
                z = z[0]
            innerNames = z.dtype.names
            if innerNames is None:
                if z.size == 1:
                    res[entry][names[n]] = z.item()
                else:
                    res[entry][names[n]] = z
                continue
            nInnerNames = len(innerNames)
            for k in range(nInnerNames):
                if type(z[innerNames[k]]) == dj.blob.MatStruct:
                    res[entry][names[n]][innerNames[k]] = recArrayParsed(z[innerNames[k]])
                else:   
                    res[entry][names[n]][innerNames[k]] = z[innerNames[k]].squeeze()

    if res.size == 1:
        return res.item()
    else:
        return res.squeeze()

In [ ]:
m0 = gu.squeeze()
z0 = m[1]["pokes"][0,0]["starting_state"]

z0[0]["L"][0].size

In [ ]:
z0

In [ ]:
recArrayParsed(z0)

In [ ]:
za = recArrayParsed(gu)[3]["states"]
za

In [ ]:
za

In [ ]:
m = gu.squeeze()
names = m.dtype.names
m.size

m[1][0][0,0][2].squeeze()

In [ ]:
gu.squeeze().dtype.names

In [ ]:
schema.list_tables()

In [ ]:
parsed_events = dj.create_virtual_module('bdata', 'bdata')

In [ ]:
bdata.Sessions() & "sessiondate > '2024-05-01'"

In [ ]:
sessions = dj.create_virtual_module('sessions', 'bl_bdata')

In [ ]:
(sessions.Sessions() & "sessid = 811000").fetch1("sessiondate")

In [ ]:
sessions.Sessions() & "sessiondate > '2021-01-01'" & "experimenter = 'Thomas'"

In [ ]:
mysessid = 956289

In [ ]:
import datetime

In [ ]:
ratinfo = dj.create_virtual_module('ratinfo', 'ratinfotest')

In [ ]:
ratinfo.Rats() & "experimenter = 'Thomas'"

In [ ]:

schema = dj.Schema(f"{dj.config['database.user']}_shapes") # 

@schema # 
class Rectangle(dj.Manual):
    definition = """ # (3)
    shape_id: int
    ---
    shape_height: float
    shape_width: float
    """


@schema
class Area(dj.Computed):
    definition = """
    -> Rectangle
    ---
    shape_area: float
    """
    def make(self, key):
        rectangle = (Rectangle & key).fetch1()
        Area.insert1(
            dict(
                shape_id=rectangle["shape_id"],
                shape_area=rectangle["shape_height"] * rectangle["shape_width"],
            )
        )